# Bioturbation example

This is a very simple script to demonstrate modelling of how bioturbation effects concentration of a contaminant in a layered soil model, where a certain depth of each layer is mixed with the depth of the neighboroughing layer(s) on each time step. The depth to mix is a function of earthworm density in the layer.

## Conceptual model

The soil profile is split into a number $L$ of soil layers, each with a depth $d_l$ [m] (where $l \in \{ 1, ..., L \}$) and concentration of contaminant $\text{[A]}_l$ [kg/m<sup>3</sup>]. Each pair of layers has a so-called "bioturbation rate" $k_{\text{bioturb},l:l+1}$ [s<sup>-1</sup>], which we define as:

$$
    k_{\text{bioturb},l:l+1} = \frac{m_{l:l+1}}{d_l}
$$

where $m_{l:l+1}$ [m/s] is the depth of each layer ($l$ and $l+1$) that is mixed each second due to bioturbation. Thus, for a time step of length $\delta t$, $m_{l:l+1} \delta t$ is mixed between the layers on each time step. $m_{l:l+1}$ itself might be a function of earthworm density, amongst other things.

![Conceptual model of the soil profile. $m_{l:l+1}\delta t$ is the depth of each layer to be mixed due to bioturbation on each time step of length $\delta t$.](img/conceptual-model.png)

If we assume that, on each model time step $t$, $m_{l:l+1} \delta t$ is instantaneously mixed between the layers $l$ and $l+1$, then the concentration of contaminant in layer $l$ on the time step $t+1$ is given by:

$$
    \text{[A]}_{l,t+1} = \text{[A]}_{l,t} + k_{\text{bioturb},l:l+1} \delta t \left( \text{[A]}_{l+1,t} - \text{[A]}_{l,t} \right) + k_{\text{bioturb},l-1:l} \delta t \left( \text{[A]}_{l-1,t} - \text{[A]}_{l,t} \right)
$$

Here, the term with factor $k_{\text{bioturb},l:l+1}$ represents mixing to the layer *below* the current layer $l$, whilst the term with factor $k_{\text{bioturb},l-1:l}$ represents mixing to the layer *above* the current layer.

For this demonstration, we define an arbitrary linear relationship between $m_{l:l+1}$ and earthworm density $w$. In reality, this will be informed by data.

$$
    m_{l:l+1} = 1 \times 10^{-8} w_l
$$

## Code

We can separate classes to represent the soil profile (`SoilProfile`) and each soil layer (`SoilLayer`). The latter has a `get_bioturbation_rate` method to calculate the bioturbation rate from earthworm density, whilst the former has a `bioturbation` method that loops over the soil layers and implements the bioturbation algorithm above. We also define an `equal` method that checks whether a the values in a list are almost equal to each other (which will be used later to tell us when to stop the calculation).

In [1]:
import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)


class SoilLayer:

    def __init__(self, depth, initial_conc, earthworm_density):
        """Initialise this soil layer with given depth, concentration and earthworm density"""
        self.depth = depth
        self.conc = initial_conc
        self.earthworm_density = earthworm_density

    def get_bioturbation_rate(self):
        """Calculate the bioturbation rate for this soil layer and the layer below"""
        bioturbation_rate = (self.earthworm_density * 1e-8) / self.depth
        return bioturbation_rate


class SoilProfile:

    def __init__(self, n_soil_layers, soil_layer_depth, initial_conc, earthworm_density):
        """Initialise the soil profile with given number of soil layers and parameters for those layers"""
        # Create the soil layers in this profile
        self.soil_layers = []
        for i in range(0, n_soil_layers):
            self.soil_layers.append(SoilLayer(soil_layer_depth[i],
                                              initial_conc[i],
                                              earthworm_density[i]))

    def bioturbation(self, t):
        """Perform bioturbation for the length of time t by mixing calculated depth of two layers together"""
        for l, layer in enumerate(self.soil_layers):
            fraction_of_layer_to_mix = layer.get_bioturbation_rate() * t
            # Don't do for the final layer
            if l < len(self.soil_layers) - 1:
                layer.conc = layer.conc + fraction_of_layer_to_mix * (self.soil_layers[l+1].conc - layer.conc)
                self.soil_layers[l+1].conc = self.soil_layers[l+1].conc + \
                                             fraction_of_layer_to_mix * (layer.conc - self.soil_layers[l+1].conc)


def equal(list):
    """Check if list is (almost) equal"""
    diff = abs(max(list) - min(list))
    return diff < 1e-12

Now we can define some configuration variables to create our soil profile with. Let's set an equal detph of 10 cm for each soil layer, and equal earthworm density of 20 worms/m2, and an initial contaminant concentration of 4 ug/m3 in the top soil layer, and 0 in lower layers. This will enable us to see how bioturbation moves the contaminant down through the profile.

In [2]:
# Config
n_soil_layers = 4
soil_layer_depth = [0.1, 0.1, 0.1, 0.1]                       # Depth of each soil layer in the profile [m]
initial_conc = [4e-9, 0, 0, 0]                                # Initial concentration in each layer [kg/m3]
earthworm_density = [20, 20, 20, 20]                          # Earthworm density for each layer [earthworms/m]

Let's pass these to the soil profile when we create it:

In [3]:
soil_profile = SoilProfile(n_soil_layers,
                           soil_layer_depth,
                           initial_conc,
                           earthworm_density)

This soil profile will now contain the nested soil layers (`soil_profile.soil_layers`), who's properties have been set based on the config data we passed it. We now need to run the bioturbation calculation over a given time period. Let's set the time step length to 1 day (86400 s) and run the simulation until steady state (using the previously defined `equal` method to check when all layers are the concentration).

In [4]:
# Fill data fill the initial concentrations (specified as input)
data = [[layer.conc] for layer in soil_profile.soil_layers]
data_t = [layer.conc for layer in soil_profile.soil_layers]
t = 0

# Perform bioturbation until concentration equal across the soil layers
while not equal(data_t):
    soil_profile.bioturbation(86400)
    for l, layer in enumerate(soil_profile.soil_layers):
        data[l].append(layer.conc)
        data_t = [layer.conc for layer in soil_profile.soil_layers]
    t += 1

The resulting data is now stored in the `data` variable. We can use [Plotly](https://plot.ly) to create a graph of the concentration for each layer.

In [5]:
py_data = [go.Scatter(y=data_l,
                      name='Layer {0}'.format(l+1)) for l, data_l in enumerate(data)]

py.iplot(go.Figure(data=py_data,
                  layout=go.Layout(xaxis={'title': 'Time (days)'},
                                   yaxis={'title': 'Concentration (kg/m3)'})))

Let's change the input data so that there is a gradient of contanimant concentration, which is highest in the deepest soil layer:

In [6]:
initial_conc = [0, 1e-9, 2e-9, 4e-9]
soil_profile = SoilProfile(n_soil_layers,
                           soil_layer_depth,
                           initial_conc,
                           earthworm_density)

Re-running the simulation and plotting the results:

In [7]:
# Fill data fill the initial concentrations (specified as input)
data = [[layer.conc] for layer in soil_profile.soil_layers]
data_t = [layer.conc for layer in soil_profile.soil_layers]
t = 0

# Perform bioturbation until concentration equal across the soil layers
while not equal(data_t):
    soil_profile.bioturbation(86400)
    for l, layer in enumerate(soil_profile.soil_layers):
        data[l].append(layer.conc)
        data_t = [layer.conc for layer in soil_profile.soil_layers]
    t += 1
    
py_data = [go.Scatter(y=data_l,
                      name='Layer {0}'.format(l+1)) for l, data_l in enumerate(data)]

py.iplot(go.Figure(data=py_data,
                  layout=go.Layout(xaxis={'title': 'Time (days)'},
                                   yaxis={'title': 'Concentration (kg/m3)'})))